calculate how many group, the number of vulnerabilities under every group, the total statistics of vuls --- serve for deployment of detection

collect the information about:
- type, location, function, attack_type, trigger_mechanism, cwe/cve

In [1]:
from pathlib import Path
import pandas as pd
import json

In [37]:
# define process for npm
def load_json_file(json_file: Path) -> json:
    with json_file.open("r") as fr:
        return json.load(fr)
    
def type_check(json_file: Path) -> str:
    with json_file.open("r") as fr:
        aff = json.load(fr)['affected'][0]
        ecosystem = aff['package']['ecosystem']
    pkg_type = ""
    if "PyPI" in ecosystem:
        pkg_type = 'python'
    elif "crates" in ecosystem:
        pkg_type = 'rust'
    elif "npm" in ecosystem:
        pkg_type = 'npm'
    elif "RubyGems" in ecosystem:
        pkg_type = 'ruby'
    else:
        print("{} has not been included".format(ecosystem))

    return pkg_type

In [44]:

def npm_json_process(json_file: Path, ecosystem):
    data = load_json_file(json_file)
    pkg_type = ecosystem
    desc = data['details']
    if "fully compromised" in desc:
        attack_type="data exfiltration/root shell"
        function='install powerful malware'
        trigger_mechanism = "download or running"
        location="entrypoint/download"
    else:
        print("There is no process for this malicious package")
        print(desc)
    hide='N/A'
    return pkg_type, location, function, attack_type, trigger_mechanism, hide


# define process for ruby
def ruby_json_process(json_file: Path, ecosystem):
    data = load_json_file(json_file)
    pkg_type = ecosystem
    desc = data['details']
    if 'communicate' in desc and "domain" in desc:
        attack_type="C2 communication"
        function="build communication tunnel with C2"
        trigger_mechanism="running connection connection"
        location="socket or other communication protocols modules"
    else:
        print("There is no process for this malicious package")
        print(desc)
    hide="N/A"
    return pkg_type, location, function, attack_type, trigger_mechanism, hide


# define process for rust
def rust_json_process(json_file: Path, ecosystem):
    data = load_json_file(json_file)
    pkg_type = ecosystem
    desc = data['details']
    if 'typosquat' in desc and "malware" in desc:
        # malware inside description --- typosquatting
        attack_type="typosquatting/malware"
        function='download payload'
    elif "communicate" in desc and "domain" in desc:
        attack_type="C2 communication" 
        function='build communication tunnel with C2'
    else:
        print("There is no process for this malicious package")
        print(desc)

    if data['id'] == "MAL-2022-1":
        trigger_mechanism="function call/condition/variable configuration"
        location='specific function'

    if data['id'] == "MAL-2023-8429":
        trigger_mechanism="running network connection"
        location='socket or other communication protocols modules'
    hide='N/A'
    return pkg_type, location, function, attack_type, trigger_mechanism, hide

# define process for public articles


In [49]:
# define process for pypi
def pypi_json_process(json_file: Path, ecosystem):
    data = load_json_file(json_file)
    pkg_type = ecosystem
    desc = data["details"]
    if "extension" in desc and "crypto" in desc and "clipboard" in desc:
        attack_type="typosquatting"
        function="access the clipboard, replay crypto wallet address"
        hide="obfuscation"
        trigger_mechanism="installation"
        location="variable names in Chinese"

    elif "spyware" in desc and "information" in desc:
        attack_type="dropper/malware"
        function="spyware and information stealing"
        hide="base64-encoded"
        trigger_mechanism="execute automatically upon download"
        location="setup"
    
    else:
        print("There is no process for this malicious package")
        print(desc)

    return pkg_type, location, function, attack_type, trigger_mechanism, hide


In [50]:
# iterate the json under per folder
data_folder = Path.cwd().parent.joinpath("data","malicious")
data_metrics = {
    "pkg_type": [],
    "location": [],
    "function": [],
    "attack_type":[],
    "trigger_mechanism":[],
    "hide":[]
}

# list_of_functions = [
#     npm_json_process,
#     ruby_json_process,
#     rust_json_process,
#     pypi_json_process
# ]

for number, json_file in enumerate(data_folder.rglob("**/*.json")):
    ecosystem = json_file.parents[1].name
    if ecosystem == 'npm':
        results = npm_json_process(json_file, ecosystem)
    elif ecosystem == 'crates-io':
        results = rust_json_process(json_file, 'rust')
    elif ecosystem == "pypi":
        results = pypi_json_process(json_file, ecosystem)
    elif ecosystem == 'rubygems':
        results = ruby_json_process(json_file, 'ruby')
    else:
        print("there is no available package process for {}".format(ecosystem))
    data_metrics['pkg_type'].append(results[0])
    data_metrics['location'].append(results[1])
    data_metrics['function'].append(results[2])
    data_metrics['attack_type'].append(results[3])
    data_metrics['trigger_mechanism'].append(results[4])
    data_metrics['hide'].append(results[5])


There is no process for this malicious package

---
_-= Per source details. Do not edit below this line.=-_

## Source: checkmarx (8c33f6b28da216b43120a3b8a8537d0263dc1eb2b22979a4183b371ff57b9e0b)
Malicious Typosquatting packages campaign targeting developers, steals cloud service credentials

## Source: google-open-source-security (a47b0bcf41b0d36b78b3429f9f22415630b3870da18554d21c2123212bc992f4)
Attack targeted at users of Alibaba, AWS and Telegram via malicious packages published to PyPI.

The malicious code was hidden in strategicly chosen functions and would only trigger when these
functions were called. The malicious code does not automatically run on install or import,
helping the packages evade detection.



UnboundLocalError: local variable 'location' referenced before assignment

In [6]:
data_metrics

{'pkg_type': [],
 'location': [],
 'function': [],
 'attack_type': [],
 'trigger_mechanism': [],
 'hide': []}